In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
import pandas as pd
import numpy as np
import keras
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, LeakyReLU
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
import joblib

In [3]:
ipl = pd.read_csv("/content/drive/MyDrive/A new project/IPL score prediction/ipl_dataset.csv")

In [4]:
df = ipl.drop(['date', 'runs', 'wickets', 'overs', 'runs_last_5', 'wickets_last_5','mid', 'striker', 'non-striker'], axis = 1)
X = df.drop(['total'], axis=1)
y = df['total']

In [5]:
venue_encoder = LabelEncoder()
batting_team_encoder = LabelEncoder()
bowling_team_encoder = LabelEncoder()
striker_encoder = LabelEncoder()
bowler_encoder = LabelEncoder()


X['venue'] = venue_encoder.fit_transform(X['venue'])
X['bat_team'] = batting_team_encoder.fit_transform(X['bat_team'])
X['bowl_team'] = bowling_team_encoder.fit_transform(X['bowl_team'])
X['batsman'] = striker_encoder.fit_transform(X['batsman'])
X['bowler'] = bowler_encoder.fit_transform(X['bowler'])

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
print(X_train.columns)
scaler = MinMaxScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
print(X_train_scaled.shape)

Index(['venue', 'bat_team', 'bowl_team', 'batsman', 'bowler'], dtype='object')
(53209, 5)


In [12]:
model = keras.Sequential([

    keras.layers.Input(shape=(X_train_scaled.shape[1],)),


    Dense(512),
    LeakyReLU(alpha=0.01),
    BatchNormalization(),
    Dropout(0.3),

    Dense(256),
    LeakyReLU(alpha=0.01),
    BatchNormalization(),
    Dropout(0.3),


    Dense(128),
    LeakyReLU(alpha=0.01),
    BatchNormalization(),
    Dropout(0.3),

    Dense(1, activation='linear')
])

huber_loss = tf.keras.losses.Huber(delta=1.0)
model.compile(optimizer=Adam(learning_rate=0.001), loss=huber_loss)

model.summary()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                      │ (None, 512)                 │           3,072 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu (LeakyReLU)              │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 512)                 │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_1 (LeakyReLU)            │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 256)                 │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_2 (LeakyReLU)            │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 128)                 │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 171,009 (668.00 KB)

 Trainable params: 169,217 (661.00 KB)

 Non-trainable params: 1,792 (7.00 KB)

In [13]:
model.fit(X_train_scaled, y_train, epochs=20, batch_size=64, validation_data=(X_test_scaled, y_test))

Epoch 1/20
832/832 ━━━━━━━━━━━━━━━━━━━━ 48s 12ms/step - loss: 139.4994 - val_loss: 21.9815
Epoch 2/20
832/832 ━━━━━━━━━━━━━━━━━━━━ 11s 14ms/step - loss: 24.0150 - val_loss: 21.0496
Epoch 3/20
832/832 ━━━━━━━━━━━━━━━━━━━━ 8s 10ms/step - loss: 23.4983 - val_loss: 20.7504
Epoch 4/20
832/832 ━━━━━━━━━━━━━━━━━━━━ 10s 12ms/step - loss: 23.2307 - val_loss: 20.2710
Epoch 5/20
832/832 ━━━━━━━━━━━━━━━━━━━━ 11s 13ms/step - loss: 22.9921 - val_loss: 20.1732
Epoch 6/20
832/832 ━━━━━━━━━━━━━━━━━━━━ 20s 12ms/step - loss: 22.5481 - val_loss: 19.8794
Epoch 7/20
832/832 ━━━━━━━━━━━━━━━━━━━━ 10s 12ms/step - loss: 22.4028 - val_loss: 19.8834
Epoch 8/20
832/832 ━━━━━━━━━━━━━━━━━━━━ 18s 10ms/step - loss: 22.1091 - val_loss: 19.6235
Epoch 9/20
832/832 ━━━━━━━━━━━━━━━━━━━━ 10s 12ms/step - loss: 21.9040 - val_loss: 19.4802
Epoch 10/20
832/832 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - loss: 21.7755 - val_loss: 19.1556
Epoch 11/20
832/832 ━━━━━━━━━━━━━━━━━━━━ 9s 10ms/step - loss: 21.6497 - val_loss: 19.1942
Epoch 12/2

In [14]:
predictions = model.predict(X_test_scaled)

713/713 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step


In [15]:
mae = mean_absolute_error(y_test, predictions)
mse = mean_squared_error(y_test, predictions)

print(f"Mean Absolute Error: {mae}")
print(f"Mean Squared Error: {mse}")

Mean Absolute Error: 18.31148338317871
Mean Squared Error: 619.8082275390625


In [16]:
# Save the trained model
joblib.dump(model, 'ipl_score_model.pkl')

# Save the label encoders
joblib.dump(venue_encoder, 'venue_encoder.pkl')
joblib.dump(batting_team_encoder, 'batting_team_encoder.pkl')
joblib.dump(bowling_team_encoder, 'bowling_team_encoder.pkl')
joblib.dump(striker_encoder, 'striker_encoder.pkl')
joblib.dump(bowler_encoder, 'bowler_encoder.pkl')

# Save the scaler
joblib.dump(scaler, 'scaler.pkl')

['scaler.pkl']